# Explore JWST and HST images

In this notebook, we load Imviz and we use mainly the user interface to explore some JWST and HST images. The data can be downloaded at this link (images are quite large so they are not packaged into a single humongous zip file. Feel free to download as many as you like.

In [ ]:
from jdaviz import Imviz

imviz = Imviz()
imviz.show()

Here is a list of steps we can do in the UI:
- Import data
- Link data by WCS
- Blink between data
- Deselect some dataset and keep only the JWST F150W and the HST file
- Blink between the two (wow to resolution)
- Zoom and pan and blink again
- Select only the JWST longest wavelength and the HST file
- Blink between the two (wow so many objects we did not see with HST)
- Take a screenshot of one of those new objects (or your favorite object) and post in slack
- Select a region around one of those objects in the JWST image
- Recenter appropriately
- Run aperture photometry and get an estimate of the flux
- Select only JWST images and create an RGB image
- Export (or screenshot) it and post in slack.